# Analysis

## Load the data

In [101]:
import pandas as pd
import pandasql as psql
import numpy as np
from datetime import datetime
import json
import re

# By Defaults pandas shows only 20 columns, so we will set the max columns display to None
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

In [102]:
users=pd.read_json('https://raw.githubusercontent.com/Rashmi0206/User-Analysis/main/users.json',lines=True)
receipts=pd.read_json('https://raw.githubusercontent.com/Rashmi0206/User-Analysis/main/receipts.json',lines=True)
brands=pd.read_json('https://raw.githubusercontent.com/Rashmi0206/User-Analysis/main/brands.json',lines=True)
users.head()

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
1,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
2,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
3,{'$oid': '5ff1e1eacfcf6c399c274ae6'},True,{'$date': 1609687530554},{'$date': 1609687530597},consumer,Email,WI
4,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI


# Part 1
## Review Existing Unstructured Data and Diagram a New Structured Relational Data Model

## a. Users Data

Lets start  by looking at first few rows of users data, total number of rows and columns and information of the columns

In [103]:
# look at first few rows
users.head()

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
1,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
2,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
3,{'$oid': '5ff1e1eacfcf6c399c274ae6'},True,{'$date': 1609687530554},{'$date': 1609687530597},consumer,Email,WI
4,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI


In [104]:
#look at dataframe shape
users.shape

(495, 7)

In [105]:
# look at the columns and datatypes
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 495 entries, 0 to 494
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   _id           495 non-null    object
 1   active        495 non-null    bool  
 2   createdDate   495 non-null    object
 3   lastLogin     433 non-null    object
 4   role          495 non-null    object
 5   signUpSource  447 non-null    object
 6   state         439 non-null    object
dtypes: bool(1), object(6)
memory usage: 23.8+ KB


I observe that there are few dict type of columns that have null values. This will create issue in extracting values of such dict columns.
Hence I will create a UDF to check null and return none as values

In [106]:
# Create function to check nulls in the dict type columns and return none for value

def CheckAndGetNull(x, key):
    if (type(x)==dict) and (key in x):
        return x[key]
    else:
        return None

Also, looks like the timestamp is in miliseconds and has nan values too, hence i will create UDF to check that and return timestamp in seconds

In [107]:
# Create function to check null timestamp and to return seconds
def TimestampToDate(x):
    if x is None or np.isnan(x):
        return None
    else:
        return datetime.fromtimestamp(x//1000)

Now I will extract id, createddate and lastlogin values from the dictionary and also use the udfs to take care of data issues

In [108]:
users['id']= users['_id'].apply(lambda x: x['$oid'])
users['created_Date']= users['createdDate'].apply(lambda x: CheckAndGetNull(x, '$date')).apply(TimestampToDate)
users['last_Login']=users['lastLogin'].apply(lambda x: CheckAndGetNull(x, '$date')).apply(TimestampToDate)
users.head()

,_id,active,createdDate,lastLogin,role,signUpSource,state,id,created_Date,last_Login
0,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,2021-01-03 10:24:04,2021-01-03 10:25:37
1,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,2021-01-03 10:24:04,2021-01-03 10:25:37
2,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,2021-01-03 10:24:04,2021-01-03 10:25:37
3,{'$oid': '5ff1e1eacfcf6c399c274ae6'},True,{'$date': 1609687530554},{'$date': 1609687530597},consumer,Email,WI,5ff1e1eacfcf6c399c274ae6,2021-01-03 10:25:30,2021-01-03 10:25:30
4,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,2021-01-03 10:24:04,2021-01-03 10:25:37


Now I will drop the non-required columns. i also noticed some duplicates in the table which I will drop as well. This will be the finalized user table in the ER diagram

In [109]:
users=users[['id', 'active', 'created_Date', 'last_Login', 'role', 'signUpSource',
       'state']].drop_duplicates()
users.head()


,id,active,created_Date,last_Login,role,signUpSource,state
0,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 10:24:04,2021-01-03 10:25:37,consumer,Email,WI
3,5ff1e1eacfcf6c399c274ae6,True,2021-01-03 10:25:30,2021-01-03 10:25:30,consumer,Email,WI
6,5ff1e1e8cfcf6c399c274ad9,True,2021-01-03 10:25:28,2021-01-03 10:25:28,consumer,Email,WI
7,5ff1e1b7cfcf6c399c274a5a,True,2021-01-03 10:24:39,2021-01-03 10:24:39,consumer,Email,WI
9,5ff1e1f1cfcf6c399c274b0b,True,2021-01-03 10:25:37,2021-01-03 10:25:37,consumer,Email,WI


## b. Receipts data

Similarly, lets start by looking at first few rows of receipts data, total number of rows and columns and information of the columns

In [110]:
receipts.head()

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
0,{'$oid': '5ff1e1eb0a720f0523000575'},500.0,"Receipt number 2 completed, bonus point schedu...",{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687536000},{'$date': 1609687531000},500.0,{'$date': 1609632000000},5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.0,5ff1e1eacfcf6c399c274ae6
1,{'$oid': '5ff1e1bb0a720f052300056b'},150.0,"Receipt number 5 completed, bonus point schedu...",{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687488000},{'$date': 1609687483000},150.0,{'$date': 1609601083000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.0,5ff1e194b6a9d73a3a9f1052
2,{'$oid': '5ff1e1f10a720f052300057a'},5.0,All-receipts receipt bonus,{'$date': 1609687537000},{'$date': 1609687537000},NaN,{'$date': 1609687542000},NaN,5.0,{'$date': 1609632000000},1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.0,5ff1e1f1cfcf6c399c274b0b
3,{'$oid': '5ff1e1ee0a7214ada100056f'},5.0,All-receipts receipt bonus,{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687539000},{'$date': 1609687534000},5.0,{'$date': 1609632000000},4.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.0,5ff1e1eacfcf6c399c274ae6
4,{'$oid': '5ff1e1d20a7214ada1000561'},5.0,All-receipts receipt bonus,{'$date': 1609687506000},{'$date': 1609687506000},{'$date': 1609687511000},{'$date': 1609687511000},{'$date': 1609687506000},5.0,{'$date': 1609601106000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.0,5ff1e194b6a9d73a3a9f1052


In [111]:
receipts.shape

(1119, 15)

In [112]:
receipts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1119 entries, 0 to 1118
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   _id                      1119 non-null   object 
 1   bonusPointsEarned        544 non-null    float64
 2   bonusPointsEarnedReason  544 non-null    object 
 3   createDate               1119 non-null   object 
 4   dateScanned              1119 non-null   object 
 5   finishedDate             568 non-null    object 
 6   modifyDate               1119 non-null   object 
 7   pointsAwardedDate        537 non-null    object 
 8   pointsEarned             609 non-null    float64
 9   purchaseDate             671 non-null    object 
 10  purchasedItemCount       635 non-null    float64
 11  rewardsReceiptItemList   679 non-null    object 
 12  rewardsReceiptStatus     1119 non-null   object 
 13  totalSpent               684 non-null    float64
 14  userId                  

Again, there are nan values in few columns of receipts data. With help of UDFs created, I will get values for all such fields as below

In [113]:

receipts['id']= receipts['_id'].apply(lambda x: x['$oid'])
receipts['create_Date']= receipts['createDate'].apply(lambda x: CheckAndGetNull(x, '$date')).apply(TimestampToDate)
receipts['date_Scanned']= receipts['dateScanned'].apply(lambda x: CheckAndGetNull(x, '$date')).apply(TimestampToDate)
receipts['finished_Date']= receipts['finishedDate'].apply(lambda x: CheckAndGetNull(x, '$date')).apply(TimestampToDate)
receipts['modify_Date']= receipts['modifyDate'].apply(lambda x: CheckAndGetNull(x, '$date')).apply(TimestampToDate)
receipts['points_AwardedDate']= receipts['pointsAwardedDate'].apply(lambda x: CheckAndGetNull(x, '$date')).apply(TimestampToDate)
receipts['purchase_Date']= receipts['purchaseDate'].apply(lambda x: CheckAndGetNull(x, '$date')).apply(TimestampToDate)
receipts.head()

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId,id,create_Date,date_Scanned,finished_Date,modify_Date,points_AwardedDate,purchase_Date
0,{'$oid': '5ff1e1eb0a720f0523000575'},500.0,"Receipt number 2 completed, bonus point schedu...",{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687536000},{'$date': 1609687531000},500.0,{'$date': 1609632000000},5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.0,5ff1e1eacfcf6c399c274ae6,5ff1e1eb0a720f0523000575,2021-01-03 10:25:31,2021-01-03 10:25:31,2021-01-03 10:25:31,2021-01-03 10:25:36,2021-01-03 10:25:31,2021-01-02 19:00:00
1,{'$oid': '5ff1e1bb0a720f052300056b'},150.0,"Receipt number 5 completed, bonus point schedu...",{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687488000},{'$date': 1609687483000},150.0,{'$date': 1609601083000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.0,5ff1e194b6a9d73a3a9f1052,5ff1e1bb0a720f052300056b,2021-01-03 10:24:43,2021-01-03 10:24:43,2021-01-03 10:24:43,2021-01-03 10:24:48,2021-01-03 10:24:43,2021-01-02 10:24:43
2,{'$oid': '5ff1e1f10a720f052300057a'},5.0,All-receipts receipt bonus,{'$date': 1609687537000},{'$date': 1609687537000},NaN,{'$date': 1609687542000},NaN,5.0,{'$date': 1609632000000},1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.0,5ff1e1f1cfcf6c399c274b0b,5ff1e1f10a720f052300057a,2021-01-03 10:25:37,2021-01-03 10:25:37,NaT,2021-01-03 10:25:42,NaT,2021-01-02 19:00:00
3,{'$oid': '5ff1e1ee0a7214ada100056f'},5.0,All-receipts receipt bonus,{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687539000},{'$date': 1609687534000},5.0,{'$date': 1609632000000},4.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.0,5ff1e1eacfcf6c399c274ae6,5ff1e1ee0a7214ada100056f,2021-01-03 10:25:34,2021-01-03 10:25:34,2021-01-03 10:25:34,2021-01-03 10:25:39,2021-01-03 10:25:34,2021-01-02 19:00:00
4,{'$oid': '5ff1e1d20a7214ada1000561'},5.0,All-receipts receipt bonus,{'$date': 1609687506000},{'$date': 1609687506000},{'$date': 1609687511000},{'$date': 1609687511000},{'$date': 1609687506000},5.0,{'$date': 1609601106000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.0,5ff1e194b6a9d73a3a9f1052,5ff1e1d20a7214ada1000561,2021-01-03 10:25:06,2021-01-03 10:25:06,2021-01-03 10:25:11,2021-01-03 10:25:11,2021-01-03 10:25:06,2021-01-02 10:25:06


* The column 'rewardsReceiptItemList' is actually as list of dictionaries. From the data dictionary, it looks like a list of all items in the a receipt and information on those items.

* From usablity perspective, it is best to create a new table from this column and also bring in the 'id' field as an identifier to join with the main receipt table.


In [114]:
receipts_item_list = receipts[['id','rewardsReceiptItemList']]

receipts= receipts[['id', 'bonusPointsEarned', 'bonusPointsEarnedReason', 'create_Date', 'date_Scanned', 'finished_Date',
       'modify_Date', 'points_AwardedDate', 'purchase_Date', 'purchasedItemCount','rewardsReceiptStatus', 'totalSpent','userId']]


In [115]:
# lets look at the info fields in the 'rewardsReceiptItemList'
receipts_item_list['rewardsReceiptItemList'].head(1).values

array([list([{'barcode': '4011', 'description': 'ITEM NOT FOUND', 'finalPrice': '26.00', 'itemPrice': '26.00', 'needsFetchReview': False, 'partnerItemId': '1', 'preventTargetGapPoints': True, 'quantityPurchased': 5, 'userFlaggedBarcode': '4011', 'userFlaggedNewItem': True, 'userFlaggedPrice': '26.00', 'userFlaggedQuantity': 5}])],
      dtype=object)

In [116]:
# since the dictionaries are in a list, we will first need to 'explode' the list to access the dict
receipts_item_list=receipts_item_list.explode('rewardsReceiptItemList')

* Now, I will extract all the dict values from the 'rewardsReceiptItemList' and create individual columns. 
* I will also add 'receipt_id' from 'id' field in receipts as joinable key.

In [117]:
receipts_item_list['receipt_id']=receipts_item_list['id']
receipts_item_list['barcode']= receipts_item_list['rewardsReceiptItemList'].apply(lambda x: CheckAndGetNull(x, 'barcode'))
receipts_item_list['description']= receipts_item_list['rewardsReceiptItemList'].apply(lambda x: CheckAndGetNull(x, 'description'))
receipts_item_list['finalPrice']= receipts_item_list['rewardsReceiptItemList'].apply(lambda x: CheckAndGetNull(x, 'finalPrice'))
receipts_item_list['itemPrice']= receipts_item_list['rewardsReceiptItemList'].apply(lambda x: CheckAndGetNull(x, 'itemPrice'))
receipts_item_list['needsFetchReview']= receipts_item_list['rewardsReceiptItemList'].apply(lambda x: CheckAndGetNull(x, 'needsFetchReview'))
receipts_item_list['partnerItemId']= receipts_item_list['rewardsReceiptItemList'].apply(lambda x: CheckAndGetNull(x, 'partnerItemId'))
receipts_item_list['preventTargetGapPoints']= receipts_item_list['rewardsReceiptItemList'].apply(lambda x: CheckAndGetNull(x, 'preventTargetGapPoints'))
receipts_item_list['quantityPurchased']= receipts_item_list['rewardsReceiptItemList'].apply(lambda x: CheckAndGetNull(x, 'quantityPurchased'))
receipts_item_list['userFlaggedBarcode']= receipts_item_list['rewardsReceiptItemList'].apply(lambda x: CheckAndGetNull(x, 'userFlaggedBarcode'))
receipts_item_list['userFlaggedNewItem']= receipts_item_list['rewardsReceiptItemList'].apply(lambda x: CheckAndGetNull(x, 'userFlaggedNewItem'))
receipts_item_list['userFlaggedPrice']= receipts_item_list['rewardsReceiptItemList'].apply(lambda x: CheckAndGetNull(x, 'userFlaggedPrice'))
receipts_item_list['userFlaggedQuantity']= receipts_item_list['rewardsReceiptItemList'].apply(lambda x: CheckAndGetNull(x, 'userFlaggedQuantity'))

Now I will drop the non-required columns. There are no duplicates in the table. This will be the finalized receipt_item_list table in the ER diagram

In [118]:
#lets keep the required fields in the receipts_item_list
receipts_item_list=receipts_item_list_final[['receipt_id','barcode', 'description', 'finalPrice',
       'itemPrice', 'needsFetchReview', 'partnerItemId',
       'preventTargetGapPoints', 'quantityPurchased', 'userFlaggedBarcode',
       'userFlaggedNewItem', 'userFlaggedPrice', 'userFlaggedQuantity']]

In [119]:
receipts_item_list.head()

,receipt_id,barcode,description,finalPrice,itemPrice,needsFetchReview,partnerItemId,preventTargetGapPoints,quantityPurchased,userFlaggedBarcode,userFlaggedNewItem,userFlaggedPrice,userFlaggedQuantity
0,5ff1e1eb0a720f0523000575,4011,ITEM NOT FOUND,26.00,26.00,False,1,True,5.0,4011,True,26.00,5.0
1,5ff1e1bb0a720f052300056b,4011,ITEM NOT FOUND,1,1,None,1,None,1.0,None,None,None,NaN
1,5ff1e1bb0a720f052300056b,028400642255,DORITOS TORTILLA CHIP SPICY SWEET CHILI REDUCE...,10.00,10.00,True,2,True,1.0,028400642255,True,10.00,1.0
2,5ff1e1f10a720f052300057a,None,None,None,None,False,1,True,NaN,4011,True,26.00,3.0
3,5ff1e1ee0a7214ada100056f,4011,ITEM NOT FOUND,28.00,28.00,False,1,True,4.0,4011,True,28.00,4.0


Similary, lets keep the required columns in the main Receipts dataframe. There are no duplicates. This is the receipts table in the ER diagram

In [120]:
receipts= receipts[['id', 'bonusPointsEarned', 'bonusPointsEarnedReason', 'create_Date', 'date_Scanned', 'finished_Date',
       'modify_Date', 'points_AwardedDate', 'purchase_Date', 'purchasedItemCount','rewardsReceiptStatus', 'totalSpent','userId']]


In [121]:
receipts.shape

(1119, 13)

In [122]:
receipts.drop_duplicates().shape

(1119, 13)

## c. Brands data

Similar to other datasets, I explored the brand data

In [123]:
brands.head()

,_id,barcode,category,categoryCode,cpg,name,topBrand,brandCode
0,{'$oid': '601ac115be37ce2ead437551'},511111019862,Baking,BAKING,"{'$id': {'$oid': '601ac114be37ce2ead437550'}, ...",test brand @1612366101024,0.0,NaN
1,{'$oid': '601c5460be37ce2ead43755f'},511111519928,Beverages,BEVERAGES,"{'$id': {'$oid': '5332f5fbe4b03c9a25efd0ba'}, ...",Starbucks,0.0,STARBUCKS
2,{'$oid': '601ac142be37ce2ead43755d'},511111819905,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146176,0.0,TEST BRANDCODE @1612366146176
3,{'$oid': '601ac142be37ce2ead43755a'},511111519874,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146051,0.0,TEST BRANDCODE @1612366146051
4,{'$oid': '601ac142be37ce2ead43755e'},511111319917,Candy & Sweets,CANDY_AND_SWEETS,"{'$id': {'$oid': '5332fa12e4b03c9a25efd1e7'}, ...",test brand @1612366146827,0.0,TEST BRANDCODE @1612366146827


In [124]:
brands.shape

(1167, 8)

In [125]:
brands.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1167 entries, 0 to 1166
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   _id           1167 non-null   object 
 1   barcode       1167 non-null   int64  
 2   category      1012 non-null   object 
 3   categoryCode  517 non-null    object 
 4   cpg           1167 non-null   object 
 5   name          1167 non-null   object 
 6   topBrand      555 non-null    float64
 7   brandCode     933 non-null    object 
dtypes: float64(1), int64(1), object(6)
memory usage: 73.1+ KB


* Again, there are nan values, which I will deal with by using the UDfs.
* The cpg column is different since it is a dict of dict. Hence I will get 'id' value from 'oid' key

In [126]:
brands['id']= brands['_id'].apply(lambda x: CheckAndGetNull(x, '$oid'))
brands['cpg_id']= brands['cpg'].apply(lambda x: x['$id']['$oid'])
brands['cpg_ref']= brands['cpg'].apply(lambda x: CheckAndGetNull(x, '$ref'))
brands.head()

,_id,barcode,category,categoryCode,cpg,name,topBrand,brandCode,id,cpg_id,cpg_ref
0,{'$oid': '601ac115be37ce2ead437551'},511111019862,Baking,BAKING,"{'$id': {'$oid': '601ac114be37ce2ead437550'}, ...",test brand @1612366101024,0.0,NaN,601ac115be37ce2ead437551,601ac114be37ce2ead437550,Cogs
1,{'$oid': '601c5460be37ce2ead43755f'},511111519928,Beverages,BEVERAGES,"{'$id': {'$oid': '5332f5fbe4b03c9a25efd0ba'}, ...",Starbucks,0.0,STARBUCKS,601c5460be37ce2ead43755f,5332f5fbe4b03c9a25efd0ba,Cogs
2,{'$oid': '601ac142be37ce2ead43755d'},511111819905,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146176,0.0,TEST BRANDCODE @1612366146176,601ac142be37ce2ead43755d,601ac142be37ce2ead437559,Cogs
3,{'$oid': '601ac142be37ce2ead43755a'},511111519874,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146051,0.0,TEST BRANDCODE @1612366146051,601ac142be37ce2ead43755a,601ac142be37ce2ead437559,Cogs
4,{'$oid': '601ac142be37ce2ead43755e'},511111319917,Candy & Sweets,CANDY_AND_SWEETS,"{'$id': {'$oid': '5332fa12e4b03c9a25efd1e7'}, ...",test brand @1612366146827,0.0,TEST BRANDCODE @1612366146827,601ac142be37ce2ead43755e,5332fa12e4b03c9a25efd1e7,Cogs


Lets keep the required columns in the brand table. This will be the brand table in the ER diagram

In [127]:
brands=brands[['id','barcode', 'category', 'categoryCode', 'name', 'topBrand',
       'brandCode', 'cpg_id', 'cpg_ref']]
brands.head()

,id,barcode,category,categoryCode,name,topBrand,brandCode,cpg_id,cpg_ref
0,601ac115be37ce2ead437551,511111019862,Baking,BAKING,test brand @1612366101024,0.0,NaN,601ac114be37ce2ead437550,Cogs
1,601c5460be37ce2ead43755f,511111519928,Beverages,BEVERAGES,Starbucks,0.0,STARBUCKS,5332f5fbe4b03c9a25efd0ba,Cogs
2,601ac142be37ce2ead43755d,511111819905,Baking,BAKING,test brand @1612366146176,0.0,TEST BRANDCODE @1612366146176,601ac142be37ce2ead437559,Cogs
3,601ac142be37ce2ead43755a,511111519874,Baking,BAKING,test brand @1612366146051,0.0,TEST BRANDCODE @1612366146051,601ac142be37ce2ead437559,Cogs
4,601ac142be37ce2ead43755e,511111319917,Candy & Sweets,CANDY_AND_SWEETS,test brand @1612366146827,0.0,TEST BRANDCODE @1612366146827,5332fa12e4b03c9a25efd1e7,Cogs


In [128]:
brands.shape

(1167, 9)

In [129]:
brands.drop_duplicates().shape

(1167, 9)

# Part 2
## Write a query that directly answers a predetermined question from a business stakeholder

From the data dictionary provided, the most reasonable way to connect brand table with receipts is through bracode fields in brand table and barcode field in receipt_item_list table. There are no other item identifier to connect brand with item and then back to receipt.
However there are issues as listed below:

* The length of barcode in brand table is 12 , but the length of barcode field in receipt_item_list is varying. Only 523 barcodes have length 12:

In [130]:
# count of barcodes by barcode length in receipt_item_list
print(psql.sqldf("select count(distinct(barcode)) as count_of_barcodes,length(barcode) as barcode_length from receipts_item_list group by length(barcode) "))


   count_of_barcodes  barcode_length
0                  0             NaN
1                  1             2.0
2                 29             4.0
3                  2             5.0
4                  4            10.0
5                  6            11.0
6                523            12.0
7                  3            13.0


In [131]:
# count of barcodes by barcode length in brand
print(psql.sqldf("select distinct length(barcode) as barcode_length from brands "))

   barcode_length
0              12


* Looking deeper into the barcodes in brand, there are 1160 unique barcodes all of which have length 12 and start with '511111'. 
* On the other hand, only 16 of 569 unique barcodes in receipt_detail_list start with '511111'

In [132]:
#unique barcodes in brands
print(psql.sqldf("select count(distinct barcode) barcode from brands"))


   barcode
0     1160


In [133]:
# unique barcodes from receipts_item_list
print(psql.sqldf("select count(distinct barcode) from receipts_item_list"))

   count(distinct barcode)
0                      568


In [134]:
# barcodes from brand matching barcodes from receipt_item_list
print(psql.sqldf("""select b.barcode,count(*) as Count from brands b 
                    inner join receipts_item_list rd on rd.barcode=b.barcode 
                    group by b.barcode"""))

         barcode  Count
0   511111001485     23
1   511111001768      3
2   511111003960      1
3   511111004127      4
4   511111101451      3
5   511111104186      1
6   511111104537      3
7   511111204206     11
8   511111502142      1
9   511111518044      1
10  511111602118      3
11  511111704140     14
12  511111802358     10
13  511111901587      3
14  511111902690      3
15  511111904175      5


Given the above obervation that shows that there are issues in joining brand with other tables since the barcode in receipt_item_list is not accurate, i will choose to work on below questions for business.

* When considering average spend from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?
* When considering total number of items purchased from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?

In [135]:
# get unique status type
print(psql.sqldf("""select distinct rewardsReceiptStatus from receipts"""))

  rewardsReceiptStatus
0             FINISHED
1             REJECTED
2              FLAGGED
3            SUBMITTED
4              PENDING


I am assuming that status as 'Finished' is accepted and 'Rejected' as rejected. Other status are not finalized yet.

In [136]:
# get avg spend and item count by status
print(psql.sqldf("""select round(avg(totalSpent),2) as AvgSpend,sum(purchasedItemCount)as TotalItems,rewardsReceiptStatus  
                    from receipts
                    where rewardsReceiptStatus in ('FINISHED','REJECTED')
                    group by rewardsReceiptStatus"""))

   AvgSpend  TotalItems rewardsReceiptStatus
0     80.85      8184.0             FINISHED
1     23.33       173.0             REJECTED


From the result, it looks like Avg spend as well as total number of items purchased of Finished is greater

# Part 3

## Evaluate Data Quality Issues in the Data Provided

Apart from the barcode issue described above, lets look at other data quality issues that can be problematic:

* Data Quality issue 1: 117 userId in receipt table are not present in users table

In [137]:
# userId in receipt table not present in users table
print(psql.sqldf("""select count(distinct userId)
                    from receipts r
                    where r.userId not in (select distinct id from users)

                    """))

   count(distinct userId)
0                     117


In [138]:
# last login is earlier than user created date
print(psql.sqldf("""select count(*)
                    from users where last_login < created_Date
                    """))

   count(*)
0         0


* Data Quality issue 2: Item purchase date is later than receipt scan date.

In [139]:
# purchase date is later than receipt create Date
print(psql.sqldf("""select purchase_Date,date_Scanned
                    from receipts where purchase_Date > date_Scanned
                    """))

                 purchase_Date                date_Scanned
0   2021-02-03 10:24:38.000000  2021-01-03 10:24:38.000000
1   2021-02-03 10:24:35.000000  2021-01-03 10:24:34.000000
2   2021-02-05 15:39:00.000000  2021-01-05 15:39:00.000000
3   2021-02-07 11:50:41.000000  2021-01-07 11:50:41.000000
4   2021-02-08 10:02:10.000000  2021-01-08 10:02:09.000000
5   2021-02-11 15:26:56.000000  2021-01-11 15:26:56.000000
6   2021-02-13 11:59:26.000000  2021-01-13 11:59:26.000000
7   2021-02-13 11:59:29.000000  2021-01-13 11:59:29.000000
8   2021-02-14 18:33:17.000000  2021-01-14 18:33:16.000000
9   2021-02-20 14:41:10.000000  2021-01-20 14:41:10.000000
10  2021-02-28 13:55:58.000000  2021-01-29 13:55:57.000000
11  2021-03-01 11:41:13.000000  2021-02-01 11:41:13.000000
12  2021-03-08 12:37:13.000000  2021-02-08 12:37:13.000000


* Data Quality issue 3: A large percentage of items (160) that are puchased dont have description.

In [140]:
print(psql.sqldf("""select description,count(*)
                    from receipts_item_list rl
                    left join receipts r on r.id=rl.receipt_id
                    group by description,barcode
                    having count(*)>20
                    order by count(*) desc
                    """))

                                          description  count(*)
0                                                None       590
1                                      ITEM NOT FOUND       160
2                             KLARBRUNN 12PK 12 FL OZ       120
3   HUGGIES SIMPLY CLEAN PREMOISTENED WIPE FRAGRAN...        92
4                        MILLER LITE 24 PACK 12OZ CAN        90
5   KLEENEX POP UP RECTANGLE BOX FACIAL TISSUE 2 P...        87
6            REGULAR PEPSI SODA COLA CAN 12 CT 144 OZ        76
7                                        CHEESE SAUCE        74
8                                           COMP BOOK        73
9   Ben & Jerry's Chunky Monkey Non-Dairy Frozen D...        60
10  KNORR RICE SIDES CREAMY CHICKEN REGULAR RICE A...        54
11  BEN & JERRYS FROZEN CHUNKY MONKEY ICE CREAM RE...        53
12                     EMIL' S SAUSAGE MUSHROOM PIZZA        50
13                                     PC PCKT WALLET        50
14                                      

* Data Quality issue 4: 54 brand code shows the barcode value.

In [141]:
# Brand Code is missing and instead showing barcode
print(psql.sqldf("""select distinct(brandCode),barcode
                    from brands
                    where brandCode like '5111%'
                    """))

       brandCode       barcode
0   511111205012  511111205012
1   511111105329  511111105329
2   511111505365  511111505365
3   511111805854  511111805854
4   511111305569  511111305569
5   511111505716  511111505716
6   511111005216  511111005216
7   511111005148  511111005148
8   511111605829  511111605829
9   511111805786  511111805786
10  511111605058  511111605058
11  511111705161  511111705161
12  511111405818  511111405818
13  511111105763  511111105763
14  511111005421  511111005421
15  511111804994  511111804994
16  511111705000  511111705000
17  511111705727  511111705727
18  511111805137  511111805137
19  511111605102  511111605102
20  511111105114  511111105114
21  511111105046  511111105046
22  511111505150  511111505150
23  511111305286  511111305286
24  511111805342  511111805342
25  511111805571  511111805571
26  511111205388  511111205388
27  511111705444  511111705444
28  511111605775  511111605775
29  511111104971  511111104971
30  511111305125  511111305125
31  5111

* Data Quality issue 5: 234 brandcodes are null

In [142]:
# Brand Code is missing and instead showing barcode
print(psql.sqldf("""select count(*)
                    from brands
                    where brandCode is null
                    """))

   count(*)
0       234


In [145]:
# Receipts with large number of items purchased(more than 50)
print(psql.sqldf("""select receipt_id, count(*)
                    from receipts_item_list
                    group by receipt_id
                    having count(*)> 50
                    order by count(*) desc
                    """))

                  receipt_id  count(*)
0   600f2fc80a720f0535000030       459
1   600f39c30a7214ada2000030       450
2   600f24970a720f053500002f       381
3   600f0cc70a720f053500002c       217
4   600a1a8d0a7214ada2000008       203
5   60049d9d0a720f05f3000094       194
6   60025cb80a720f05f300008d       185
7   600260210a720f05f300008f       183
8   600a1e270a720f0535000009       176
9   600edb570a720f053500001d       155
10  600b139e0a7214ada200000a       154
11  600370250a720f05f3000090       148
12  6004a5f20a7214ad4c000088       147
13  600b34ca0a7214ada200000c       146
14  600373700a720f05f3000091       141
15  60023e8f0a720f05f300008b       137
16  600eda0c0a7214ada200001e       131
17  6002590c0a7214ad4c000082       130
18  600996ac0a720f05fa000134       127
19  6004a99e0a720f05f3000095       126
20  600ba6ae0a7214ada2000010       125
21  600b39f60a720f053500000e       125
22  60099c3c0a7214ad89000135       124
23  600b420b0a7214ada200000d       123
24  60088a100a7214ad89000